# Esercizio 2

Accedere a https://www.kaggle.com/rounakbanik/the-movies-dataset#movies_metadata.csv e scaricare il dataset in formato csv `movies_metadata.csv.zip`. Tale dataset contiene, in *record* di 24 campi, i film realizzati fino al 2017.

Considerando solo i primi 1000 record, si richiede di:
- elencare i 10 paesi che hanno prodotto più film, ciascuno con il numero di film prodotti (ordinati per numero di film prodotti decrescente)
- per ognuno dei generi cinematografici presenti:
    - la classifica degli n film più popolari ordinati per popolarità decrescente, specificando il titolo originale e la tagline per ciascuno di essi
    - l'elenco delle lingue originali che sono coinvolte in ciascuna delle classifiche del punto precedente

Parametri di input:

- nome del dataset in input
- parametro n

***

I campi del file csv che occorrono per risolvere l'esercizio sono:

- `genres` (stringa)

        [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

- `original_title` (stringa)
- `popularity` (decimale)
- `id` (ID)
- `tagline` (stringa)
- `original_language` (stringa)
- `production_countries` (stringa)

        [{'iso_3166_1': 'DE', 'name': 'Germany'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]

***

Requisiti generali:

- le stringhe relative ai campi `genres` e `production_countries` *letteralmente* specificano una lista di dizionari in cui è presente la chiave `name` a cui viene associato il valore di un genere cinematografico nel caso di `genres` e un paese nel caso di `production_countries`. Si richiede quindi la definizione di una funzione `get_items()` che prenda in input uno qualsiasi tra questi due campi ed estragga la lista dei generi (ad esempio `['Animation', 'Comedy', 'Family']`) nel caso di `genres` e la lista dei paesi di produzione (ad esempio `['Germany', 'United States of America']`) nel caso di `production_countries`.
- si richiede produrre le seguenti variabili:
    - `country_rank_list`: lista di 10 tuple *(paese, numero di film prodotti)* contenente i primi 10 paesi che hanno prodotto più film (ordinati per numero di film prodotti decrescente)
- `pop_rank_dict`: dizionario delle classifiche per genere dei primi n film ordinati per popolarità decrescente
    - *chiave*: genere di un film
    - *valore*: lista di n liste *[titolo originale, tagline]* contenente i primi n film ordinati per popolarità decrescente
- `language_set_dict`: dizionario degli insiemi delle lingue coinvolte in ciascuna delle classifiche presenti in `pop_rank_dict`
    - *chiave*: genere di un film
    - *valore*: insieme delle lingue coinvolte


**NOTA BENE**: alcuni record del dataset csv hanno un valore del campo `popularity` che non è un decimale oppure non hanno proprio un valore. Non conviene perciò considerarle.

***

## Soluzione

### Parametri in input

In [1]:
input_file_name = './movies_metadata.csv.zip'
n = 15

Importazione dei moduli `pandas`, `ast` e `numpy`.

### Definizione della funzione `get_items()`

La funzione `get_items()` estrae la lista dei generi/paesi dai campi `genres`/`production_countries` del dataset csv

### Lettura del dataset csv in un *data frame* `movies_df`

Leggere il dataset csv ed estrarre solo le prime 1000 righe.

In [5]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Costruzione delle strutture dati di base

Costruire le seguenti strutture di base:
- `info_dict`: dizionario delle informazioni sui film
    - *chiave*: ID dei film (campo `id` del dataset csv)
    - *valore*: tupla *(titolo originale, tagline, lingua originale)*
- `country_list`: lista dei paesi che hanno prodotto un film (ciascun paese deve essere presente nella lista esattamente per il numero di volte in cui ha prodotto un film)
- `pop_dict`: dizionario delle popolarità
    - *chiave*: genere cinematografico
    - *valore*: lista dei film associati al genere (ognuno dei film è presente come lista annidata *[popolarità, ID]*

Inizializzare le strutture di base vuote.

Attraversare i *record* di `movies_df` per riempire `info_dict` e `country_list`, e costruire `pop_dict` con valori tutti uguali a liste vuote `[]`.

**NOTA BENE**: effettuare il controllo sul valore del campo `popularity` che deve essere un numero (altrimenti non considerare il record).

In [8]:
info_dict

{'862': ('Toy Story', nan, 'en'),
 '8844': ('Jumanji', 'Roll the dice and unleash the excitement!', 'en'),
 '15602': ('Grumpier Old Men',
  'Still Yelling. Still Fighting. Still Ready for Love.',
  'en'),
 '31357': ('Waiting to Exhale',
  'Friends are the people who let you be yourself... and never let you forget it.',
  'en'),
 '11862': ('Father of the Bride Part II',
  "Just When His World Is Back To Normal... He's In For The Surprise Of His Life!",
  'en'),
 '949': ('Heat', 'A Los Angeles Crime Saga', 'en'),
 '11860': ('Sabrina',
  'You are cordially invited to the most surprising merger of the year.',
  'en'),
 '45325': ('Tom and Huck', 'The Original Bad Boys.', 'en'),
 '9091': ('Sudden Death', 'Terror goes into overtime.', 'en'),
 '710': ('GoldenEye', 'No limits. No fears. No substitutes.', 'en'),
 '9087': ('The American President',
  "Why can't the most powerful man in the world have the one thing he wants most?",
  'en'),
 '12110': ('Dracula: Dead and Loving It', nan, 'en'),
 '2

In [9]:
country_list

['United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'Germany',
 'United States of America',
 'United States of America',
 'United States of America',
 'United Kingdom',
 'United States of America',
 'United States of America',
 'France',
 'United States of America',
 'United States of America',
 'United States of America',
 'France',
 'Germany',
 'Italy',
 'United States of America',
 'France',
 'United States of America',
 'United Kingdom',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'France',
 'United States of America',
 'United States of America',
 'United States of America',
 'Italy',
 'United States of America',
 'United Kingdom',
 'France',
 'France',
 'Germany',
 'Spain',
 'China',
 'France',
 'United States of America',
 'Unite

In [10]:
pop_dict

{'Animation': [],
 'Comedy': [],
 'Family': [],
 'Adventure': [],
 'Fantasy': [],
 'Romance': [],
 'Drama': [],
 'Action': [],
 'Crime': [],
 'Thriller': [],
 'Horror': [],
 'History': [],
 'Science Fiction': [],
 'Mystery': [],
 'War': [],
 'Foreign': [],
 'Music': [],
 'Documentary': [],
 'Western': [],
 'TV Movie': []}

Attraversare di nuovo i *record* di `movies_df` per riempire i valori di `pop_dict`.

In [12]:
pop_dict

{'Animation': [[21.946943, '862'],
  [12.140733, '21032'],
  [13.280069000000001, '10530'],
  [10.177977, '15789'],
  [0.090452, '43475'],
  [8.910461999999999, '22586'],
  [21.605760999999998, '8587'],
  [0.381704, '18242'],
  [17.730913, '9479'],
  [7.436000999999999, '15139'],
  [16.357419, '812'],
  [16.263878, '408'],
  [23.433511, '10020'],
  [14.202283999999999, '10895'],
  [7.640072999999999, '11827'],
  [9.834047, '10112'],
  [5.028897, '19042'],
  [13.217623000000001, '10539'],
  [11.054250999999999, '2300'],
  [10.813578999999999, '12233'],
  [0.543042, '40651'],
  [10.686166, '9323'],
  [6.796858, '532'],
  [13.553999, '10545'],
  [7.523528, '18975'],
  [6.3724419999999995, '19004'],
  [14.436857999999999, '11224'],
  [1.054814, '81310'],
  [8.702472, '15947'],
  [10.636458, '9078'],
  [0.901443, '29682']],
 'Comedy': [[21.946943, '862'],
  [11.7129, '15602'],
  [3.859495, '31357'],
  [8.387519000000001, '11862'],
  [6.677277, '11860'],
  [6.318445, '9087'],
  [5.430331, '1

### Estrazione dei 10 paesi che hanno prodotto più film

Costruire la lista `country_rank_list` dei primi 10 paesi che hanno prodotto più film, ordinati per numero di film decrescente; ogni paese deve essere presente come tupla *(paese, numero di film prodotti)*.

In [14]:
country_rank_list

[('United States of America', 742),
 ('United Kingdom', 102),
 ('France', 96),
 ('Germany', 45),
 ('Canada', 30),
 ('Italy', 29),
 ('Japan', 21),
 ('Australia', 12),
 ('Hong Kong', 12),
 ('Ireland', 11)]

### Estrazione, per ogni genere cinematografico, degli n film più popolari ordinati per popolarità descrescente, ed estrazione delle lingue coinvolte in ciascuna classifica di popolarità

Ottenere da `pop_dict` il dizionario `pop_rank_dict_temp` con la stessa struttura, ma con la differenza che il *valore* relativo a un dato genere è la lista degli n film più popolari (ordinati per popolarità decrescente).

**NOTA BENE**: ottenere prima `key_value_tuples`, lista di tuple *(chiave, valore)* di `pop_rank_dict_temp`, e in seguito costruire il dizionario con la funzione `dict()`.

In [16]:
pop_rank_dict_temp

{'Animation': [[23.433511, '10020'],
  [21.946943, '862'],
  [21.605760999999998, '8587'],
  [17.730913, '9479'],
  [16.357419, '812'],
  [16.263878, '408'],
  [14.436857999999999, '11224'],
  [14.202283999999999, '10895'],
  [13.553999, '10545'],
  [13.280069000000001, '10530'],
  [13.217623000000001, '10539'],
  [12.140733, '21032'],
  [11.054250999999999, '2300'],
  [10.813578999999999, '12233'],
  [10.686166, '9323']],
 'Comedy': [[48.307194, '13'],
  [21.946943, '862'],
  [17.555909, '2758'],
  [17.136672, '8005'],
  [16.357419, '812'],
  [16.236745000000003, '10525'],
  [15.899134, '11448'],
  [15.783615, '2064'],
  [15.534707000000001, '31586'],
  [15.11549, '433'],
  [15.050326, '9359'],
  [15.019585999999999, '888'],
  [14.657754, '25059'],
  [14.644772, '11853'],
  [14.587894, '11902']],
 'Family': [[23.433511, '10020'],
  [22.064932000000002, '630'],
  [21.946943, '862'],
  [21.605760999999998, '8587'],
  [17.730913, '9479'],
  [17.555909, '2758'],
  [17.015539, '8844'],
  [

Ottenere da `pop_rank_dict_temp` il dizionario `pop_rank_dict`
- *chiave*: genere cinematografico
- *valore*: lista degli n film più popolari, dove ogni film è presente come lista annidata *[titolo originale, tagline]*

**NOTA BENE**: ottenere prima `key_value_tuples`, lista di tuple *(chiave, valore)* di `pop_rank_dict`, e in seguito costruire il dizionario con la funzione `dict()`.

In [18]:
pop_rank_dict

{'Animation': [['Beauty and the Beast',
   'The most beautiful love story ever told.'],
  ['Toy Story', nan],
  ['The Lion King',
   "Life's greatest adventure is finding your place in the Circle of Life."],
  ['The Nightmare Before Christmas',
   'A ghoulish tale with wicked humor & stunning animation.'],
  ['Aladdin', 'Wish granted!'],
  ['Snow White and the Seven Dwarfs',
   'The Happiest, Dopiest, Grumpiest, Sneeziest movie of the year.'],
  ['Cinderella', 'The greatest love story ever told.'],
  ['Pinocchio', 'For anyone who has ever wished upon a star.'],
  ['The Hunchback of Notre Dame', nan],
  ['Pocahontas', 'An American legend comes to life.'],
  ['James and the Giant Peach', "Adventures this big don't grow on trees."],
  ['Balto', 'Part Dog. Part Wolf. All Hero.'],
  ['Space Jam', 'Get ready to jam.'],
  ['Oliver & Company', 'The first Disney movie with attitude.'],
  ['GHOST IN THE SHELL', 'It found a voice... Now it needs a body.']],
 'Comedy': [['Forrest Gump',
   "The wo

Ottenere da `pop_rank_dict_temp` il dizionario `language_set_dict` degli insiemi delle lingue coinvolte in ciascuna delle classifiche per genere degli n film ordinati per popolarità descrescente
- *chiave*: genere cinematografico
- *valore*: insieme delle lingue originali coinvolte (oggetto di tipo `set`)

**NOTA BENE**: ottenere prima `key_value_tuples`, lista di tuple *(chiave, valore)* di `language_set_dict`, e in seguito costruire il dizionario con la funzione `dict()`.

In [20]:
language_set_dict

{'Animation': {'en', 'ja'},
 'Comedy': {'en', 'sr'},
 'Family': {'en'},
 'Adventure': {'en'},
 'Fantasy': {'en'},
 'Romance': {'en'},
 'Drama': {'en', 'fr'},
 'Action': {'en'},
 'Crime': {'en', 'fr'},
 'Thriller': {'en', 'fr'},
 'Horror': {'en', 'fr'},
 'History': {'de', 'en', 'fr'},
 'Science Fiction': {'en'},
 'Mystery': {'en'},
 'War': {'de', 'en', 'sr', 'zh'},
 'Foreign': {'cn', 'de', 'en', 'es', 'he', 'it'},
 'Music': {'en', 'fr'},
 'Documentary': {'de', 'en', 'pt'},
 'Western': {'en'},
 'TV Movie': {'en'}}